In [1]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import transforms

D:\develop\tools\anaconda3\envs\rl-learn\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\develop\tools\anaconda3\envs\rl-learn\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\develop\tools\anaconda3\envs\rl-learn\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


OSError: [WinError 1455] 이 작업을 완료하기 위한 페이징 파일이 너무 작습니다. Error loading "D:\develop\tools\anaconda3\envs\rl-learn\lib\site-packages\torch\lib\cudnn_adv_train64_8.dll" or one of its dependencies.

In [ ]:
import pandas as pd

In [ ]:
pd.options.display.max_columns=500
pd.options.display.max_colwidth = None
pd.options.display.max_rows = None

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def show_rgb_image(tensor_state):
    plt.imshow(tensor_state.cpu().detach().numpy().transpose(1, 2, 0))
    
def show_bw_image(tensor_state):
    plt.imshow(tensor_state.cpu().detach().numpy())
    
    
def kernel_show(length, kernel_list, row=5):
    fig, axes = plt.subplots(int(length / row) + 1, row, figsize=(15,10))
    axes = axes.flatten()
    for i in range(length):
        axes[i].imshow(kernel_list[i].cpu().detach())
        
def list_show(l, st, end):
    fig, axes = plt.subplots(1, end - st + 1, figsize=(15,10))
    axes = axes.flatten()
    for i in range(end - st + 1):
        axes[i].imshow(l[i].cpu().detach().numpy().transpose(1, 2, 0))

In [ ]:
class Environment(gym.Wrapper):
    move = -1
    eat_cookie = 100
    eat_ghost = 0
    death = -1000

    def __init__(self):
        super(Environment, self).__init__(gym.make('MsPacman-v0'))

        self._move_reward = Environment.move
        self._eat_cookie_reward = Environment.eat_cookie
        self._death_reward = Environment.death
        self._eat_ghost_reward = Environment.eat_ghost

    def reset(self,
              move_reward: int = move,
              eat_cookie_reward: int = eat_cookie,
              death_reward: int = death,
              eat_ghost_reward: int = eat_ghost,
              **kwargs):

        self._move_reward = move_reward
        self._eat_cookie_reward = eat_cookie_reward
        self._death_reward = death_reward
        self._eat_ghost_reward = eat_ghost_reward

        self._metadata = {
            'lives': 3,
            'get_coin': 0
        }

        state = super(Environment, self).reset(**kwargs)
        return self.observation(state)

    def step(self, action):

        state_prime, reward, done, info = super(Environment, self).step(action)

        state_prime = self.observation(state_prime)
        new_reward = self.reward(reward, info)

        self._metadata['lives'] = info['lives']
                
        return state_prime, new_reward, done, info

    def reward(self, reward, info):

        new_reward = 0

        # move
        if reward == 0:
            new_reward = self._move_reward
        # eat
        elif reward == 10:
            new_reward = self._eat_cookie_reward
            self._metadata['get_coin'] += 1
        elif reward == 200:
            new_reward = self._eat_ghost_reward

        if self._metadata['lives'] > info['lives'] != 3:
            new_reward += self._death_reward * (1 - (self._metadata['get_coin'] % 150) / 150)
            self._metadata['get_coin'] = 0 # 한 목숨당 처리하도록? 처음에 많이 먹고 나중에 ㅈㄴ 놀수 있으니깐..?

        return new_reward

    def observation(self, observation):
        transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize((110, 84)),
            transforms.ToTensor()
        ])
        

        return transform(observation)[:84, :84]



In [ ]:
import gym

env = Environment()

state = env.reset()

result = []

for _ in range(10000):
    state_prime, reward, done, info = env.step(env.action_space.sample())
    
    result.append({
        'state': state,
        'state_prime': state_prime,
        'reward': reward,
        'done': done,
        'info': info
    })
    
    if done:
        break
    
    state = state_prime


In [ ]:
len(result)

In [ ]:
test_state = result[450]['state']

In [ ]:
test_state.shape

In [ ]:
test_state[0]

In [ ]:
show_rgb_image(test_state)

In [ ]:
test_state = torch.cat([result[450]['state'], result[451]['state'], result[452]['state']])

In [ ]:
rgb_state = test_state.unsqueeze(0)

In [ ]:
rgb_state.shape

In [ ]:
rgb_state[0]

### Layer1

In [ ]:
test_layer1 = nn.Sequential(
    nn.Conv2d(
        in_channels=3,
        out_channels=16,
        kernel_size=2,
        device='cpu'),
    nn.BatchNorm2d(16, device='cpu'),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2)
)

In [ ]:
layer1_result = test_layer1(rgb_state)
layer1_result.shape

In [ ]:
show_rgb_image(test_state)

In [ ]:
len(layer1_result.view(-1))

In [ ]:
kernel_show(kernel_list=layer1_result[0], length=len(layer1_result[0]), row=5)

### Layer2

In [ ]:
test_layer2 = nn.Sequential(
    nn.Conv2d(
        in_channels=3,
        out_channels=16,
        kernel_size=2,
        stride=2,
        device='cpu'),
    nn.BatchNorm2d(16, device='cpu'),
    nn.ReLU(),
    nn.Conv2d(in_channels=16,
        out_channels=32,
        kernel_size=2,
        device='cpu'),
    nn.BatchNorm2d(32, device='cpu'),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2)
)

In [ ]:
show_rgb_image(test_state)

In [ ]:
test_state.shape

In [ ]:
layer2_result = test_layer2(rgb_state)
layer2_result.shape

In [ ]:
len(layer2_result.view(-1))

In [ ]:
kernel_show(kernel_list=layer2_result[0], length=len(layer2_result[0]), row=6)

### Custom Layer

In [ ]:
test_layer3 = nn.Sequential(
    nn.Conv2d(
        in_channels=3,
        out_channels=16,
        kernel_size=2,
        device='cpu'),
    nn.BatchNorm2d(16, device='cpu'),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(in_channels=16,
        out_channels=32,
        kernel_size=2,
        device='cpu'),
    nn.BatchNorm2d(32, device='cpu'),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(in_channels=32,
        out_channels=64,
        kernel_size=2,
        device='cpu'),
    nn.BatchNorm2d(64, device='cpu'),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3),
)

In [ ]:
layer3_result = test_layer3(rgb_state)
layer3_result.shape

In [ ]:
show_rgb_image(test_state)

In [ ]:
len(layer3_result.view(-1))

In [ ]:
kernel_show(kernel_list=layer3_result[0], length=len(layer3_result[0]), row=10)